In [ ]:
#!pip install face_recognition

In [ ]:
import os
import cv2
import dlib
import mimetypes
#import face_recognition

import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/Model.h5"
model_detect_deepfake = load_model(model_path)

#Đường dẫn tới thư mục lưu các khung hình của video
direct_to_frames_folder = '/content/tmp'
os.makedirs(direct_to_frames_folder)



Mounted at /content/drive


In [ ]:
def update_status(message: str) -> None:
    print(f'{message}')
    return

def is_image(image_path: str) -> bool:
    if image_path and os.path.isfile(image_path):
        mimetype, _ = mimetypes.guess_type(image_path)
        return bool(mimetype and mimetype.startswith('image/'))
    return False

def is_video(video_path: str) -> bool:
    if video_path and os.path.isfile(video_path):
        mimetype, _ = mimetypes.guess_type(video_path)
        return bool(mimetype and mimetype.startswith('video/'))
    return False

def pre_start(source_path: str) -> bool:
    if not is_image(source_path) and not is_video(source_path):
        update_status('Select an image or video for source path.')
        return False
    return True

def has_image_extension(image_path: str) -> bool:
    return image_path.lower().endswith(('png', 'jpg', 'jpeg', 'webp'))



In [ ]:
#xử lý hình ảnh / khung hình

def check_deepfake(frame_path) -> bool:
    # Đường dẫn đến ảnh cần phân loại

    # Đọc và tiền xử lý ảnh đầu vào
    img = image.load_img(frame_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # frame = cv2.resize(frame, (224, 224))
    # frame = np.expand_dims(frame, axis=0)
    # frame = preprocess_input(frame)

    predictions = model_detect_deepfake.predict(img_array)
    label = np.argmax(predictions)
    print(predictions)

    return label

# def process_image(source_path: str) -> bool:
#     image = face_recognition.load_image_file(source_path)
#     face_locations = face_recognition.face_locations(image)

#     is_deepfake_image = False
#     for face in face_locations:
#         top, right, bottom, left = face
#         tmp_image_face = image[top:bottom, left:right]

#         if check_deepfake(tmp_image_face) == 1:
#             is_deepfake_image = True
#             break
#     return is_deepfake_image

# def process_image(source_path: str) -> bool:
#     img = image.load_img(source_path, target_size=(224, 224))
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     img_array = preprocess_input(img_array)

#     is_deepfake_image = False
#     if check_deepfake(img_array) == 1:
#         is_deepfake_image = True
#     return is_deepfake_image


In [ ]:
#xử lý video
def extracts_frames(video_path):
    video = cv2.VideoCapture(video_path)

    # Đọc và trích xuất khung hình từ video
    frame_count = 0
    while True:
        # Đọc một khung hình từ video
        ret, frame = video.read()

        # Kiểm tra xem việc đọc khung hình thành công hay không
        if not ret:
            break

        # Lưu khung hình thành ảnh
        frame_path = '{}/frame_{}.jpg'.format(direct_to_frames_folder, frame_count)
        cv2.imwrite(frame_path, frame)

        # Tăng đếm số khung hình
        frame_count += 1

    # Giải phóng tài nguyên và đóng video
    video.release()
    cv2.destroyAllWindows()

def process_frames() -> bool:
    frames_list = os.listdir(direct_to_frames_folder)
    num_frames = len(frames_list)
    num_deepfake_frames = 0

    for i in range(num_frames):
        frame_path = '{}/frame_{}.jpg'.format(direct_to_frames_folder, i)
        if check_deepfake(frame_path): num_deepfake_frames += 1

    deepfake_percentage = int(num_deepfake_frames/num_frames *100)
    print("Tỷ lệ số khung hình deepfake: {}% tương ứng với {} / {} khung hình".format(deepfake_percentage, num_deepfake_frames, num_frames))


In [ ]:
source_path = "/content/drive/MyDrive/demo.mp4"
def start() -> None:
    if not pre_start(source_path):
        return
        # source = image
    if has_image_extension(source_path):
        update_status('Progressing... image')
        if check_deepfake(source_path):
            print('Đây là hình ảnh deepfake')
        else:
            print('Đây không là hình ảnh deepfake')
    else:
        # source = video
        update_status('Progressing... video')
        extracts_frames(source_path)
        process_frames()
start()

Progressing... video
1/1 [==============================] - 1s 714ms/step
[[0.589132   0.41086805]]
1/1 [==============================] - 0s 483ms/step
[[0.6225677  0.37743235]]
1/1 [==============================] - 1s 567ms/step
[[0.5668565  0.43314347]]
1/1 [==============================] - 0s 468ms/step
[[0.49947345 0.5005265 ]]
1/1 [==============================] - 0s 450ms/step
[[0.56852525 0.4314748 ]]
1/1 [==============================] - 0s 461ms/step
[[0.606091   0.39390898]]
1/1 [==============================] - 0s 454ms/step
[[0.622489   0.37751096]]
1/1 [==============================] - 0s 460ms/step
[[0.5896551  0.41034493]]
1/1 [==============================] - 0s 451ms/step
[[0.59415406 0.40584594]]
1/1 [==============================] - 0s 455ms/step
[[0.655402   0.34459805]]
1/1 [==============================] - 0s 459ms/step
[[0.587279   0.41272104]]
1/1 [==============================] - 0s 461ms/step
[[0.5706446  0.42935535]]
1/1 [==========================

In [ ]:
source_path = "/content/drive/MyDrive/demo_fake.mp4"
def start() -> None:
    if not pre_start(source_path):
        return
        # source = image
    if has_image_extension(source_path):
        update_status('Progressing... image')
        if check_deepfake(source_path):
            print('Đây là hình ảnh deepfake')
        else:
            print('Đây không là hình ảnh deepfake')
    else:
        # source = video
        update_status('Progressing... video')
        extracts_frames(source_path)
        process_frames()
start()

Progressing... video
1/1 [==============================] - 0s 456ms/step
[[0.5514337  0.44856635]]
1/1 [==============================] - 0s 449ms/step
[[0.49651456 0.5034855 ]]
1/1 [==============================] - 0s 470ms/step
[[0.4885525  0.51144755]]
1/1 [==============================] - 0s 452ms/step
[[0.44971946 0.5502806 ]]
1/1 [==============================] - 0s 464ms/step
[[0.38107282 0.6189272 ]]
1/1 [==============================] - 0s 451ms/step
[[0.30858406 0.69141597]]
1/1 [==============================] - 0s 462ms/step
[[0.2804363 0.7195636]]
1/1 [==============================] - 0s 450ms/step
[[0.28471014 0.7152899 ]]
1/1 [==============================] - 0s 467ms/step
[[0.3317778 0.6682222]]
1/1 [==============================] - 0s 458ms/step
[[0.25542292 0.7445771 ]]
1/1 [==============================] - 0s 465ms/step
[[0.2797598 0.7202402]]
1/1 [==============================] - 0s 451ms/step
[[0.33365694 0.66634303]]
1/1 [==============================] 